Самым успешным оказался вариант 1D-CNN + Attention с 65% точностью. Возможно, если бы данных было больше, мы бы смогли получить более высокую точность используя DL методы.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Dropout, BatchNormalization,
                                     LSTM, Bidirectional, LayerNormalization)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sentence_transformers import SentenceTransformer

from sklearn.linear_model import LogisticRegression

from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

In [2]:
def fill_missing_prices(df):
    df = df.copy().reset_index(drop=True)
    for i in range(len(df)):
        if pd.isna(df.loc[i, 'Open Price']) and i > 0:
            df.loc[i, 'Open Price'] = df.loc[i-1, 'Close Price']
        if pd.isna(df.loc[i, 'Close Price']) and i < len(df)-1:
            df.loc[i, 'Close Price'] = df.loc[i+1, 'Open Price']
    return df

In [3]:
DATA_PATH = 'Apple_data.csv'
data = pd.read_csv(DATA_PATH)
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date').reset_index(drop=True)

data = fill_missing_prices(data).dropna()

In [4]:
daily = data.groupby('Date').agg(
    Open_Price=('Open Price', 'first'),
    Close_Price=('Close Price', 'last')
).reset_index()
daily['SMA_5'] = daily['Close_Price'].rolling(5).mean()
daily['SMA_10'] = daily['Close_Price'].rolling(10).mean()
daily['Return'] = daily['Close_Price'].pct_change()
daily = daily.dropna().reset_index(drop=True)

In [5]:
bert = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = (
    data.groupby('Date')['headline']
        .apply(lambda lst: bert.encode(lst.tolist(), show_progress_bar=False).mean(0))
        .reset_index()
)
emb_cols = [f'emb_{i}' for i in range(embeddings['headline'].iloc[0].shape[0])]
emb_df = pd.DataFrame(embeddings['headline'].tolist(), columns=emb_cols)
emb_daily = pd.concat([embeddings[['Date']], emb_df], axis=1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
features = pd.merge(daily, emb_daily, on='Date')
features['Target'] = (features['Close_Price'] > features['Open_Price']).astype(int)

In [7]:
X = features.drop(['Date', 'Target', 'Open_Price', 'Close_Price'], axis=1)
y = features['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model_ff = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128), BatchNormalization(), tf.keras.layers.Activation('relu'), Dropout(0.3),
    Dense(64),  BatchNormalization(), tf.keras.layers.Activation('relu'), Dropout(0.3),
    Dense(1, activation='sigmoid')
])
model_ff.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
callbacks = [
    ModelCheckpoint('ff_best.keras', save_best_only=True, monitor='val_accuracy'),
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=5)
]
history_ff = model_ff.fit(
    X_train, y_train,
    validation_split=0.2, epochs=50, batch_size=16,
    callbacks=callbacks, verbose=1
)

loss_ff, acc_ff = model_ff.evaluate(X_test, y_test)
print(f"FF Test Acc: {acc_ff:.4f}")

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.4507 - loss: 0.9346 - val_accuracy: 0.5652 - val_loss: 1.1294 - learning_rate: 0.0010
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6234 - loss: 0.6774 - val_accuracy: 0.5652 - val_loss: 2.0826 - learning_rate: 0.0010
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6012 - loss: 0.6532 - val_accuracy: 0.5652 - val_loss: 1.1323 - learning_rate: 0.0010
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6808 - loss: 0.6130 - val_accuracy: 0.5652 - val_loss: 0.6871 - learning_rate: 0.0010
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6312 - loss: 0.6286 - val_accuracy: 0.5797 - val_loss: 0.6516 - learning_rate: 0.0010
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7356 - loss: 0.5665 - val_accuracy: 0.5942 - val_loss: 0.6929 - learning_rate: 0.0010
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7236 - loss: 0.5668 - val_accurac

In [9]:
SEQ_LEN = 5
feature_cols = X.columns.tolist()
def build_sequences(X_df, y_series, seq_len):
    Xs, ys = [], []
    for i in range(seq_len, len(X_df)):
        Xs.append(X_df.iloc[i-seq_len:i].values)
        ys.append(y_series.iloc[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = build_sequences(features[feature_cols], features['Target'], SEQ_LEN)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42, shuffle=False
)

In [10]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1],), initializer='random_normal', trainable=True)
    def call(self, inputs):
        score = tf.tensordot(inputs, self.W, axes=[2,0])
        weights = tf.nn.softmax(score, axis=1)
        context = tf.reduce_sum(inputs * tf.expand_dims(weights, -1), axis=1)
        return context

inp = Input(shape=(SEQ_LEN, len(feature_cols)))
x = Bidirectional(LSTM(64, return_sequences=True))(inp)
x = LayerNormalization()(x)
attn = AttentionLayer()(x)
out = Dense(32, activation='relu')(attn)
out = Dropout(0.3)(out)
out = Dense(1, activation='sigmoid')(out)
model_lstm_attn = Model(inputs=inp, outputs=out)
model_lstm_attn.compile(
    optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy']
)

callbacks2 = [
    ModelCheckpoint('lstm_attn_best.keras', save_best_only=True, monitor='val_accuracy'),
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=5)
]

In [11]:
history_lstm = model_lstm_attn.fit(
    X_train_s, y_train_s,
    validation_split=0.2, epochs=50, batch_size=16,
    callbacks=callbacks2, verbose=1
)

loss_la, acc_la = model_lstm_attn.evaluate(X_test_s, y_test_s)
print(f"LSTM+Attn Test Acc: {acc_la:.4f}")

Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.5464 - loss: 0.7138 - val_accuracy: 0.5000 - val_loss: 0.7016 - learning_rate: 0.0010
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5426 - loss: 0.6972 - val_accuracy: 0.5000 - val_loss: 0.6961 - learning_rate: 0.0010
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.4773 - loss: 0.7059 - val_accuracy: 0.5000 - val_loss: 0.6989 - learning_rate: 0.0010
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4928 - loss: 0.6865 - val_accuracy: 0.5294 - val_loss: 0.6923 - learning_rate: 0.0010
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4984 - loss: 0.6959 - val_accuracy: 0.5294 - val_loss: 0.6938 - learning_rate: 0.0010
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5323 - loss: 0.6828 - val_accuracy: 0.5000 - val_loss: 0.6943 - learning_rate: 0.0010
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5418 - loss: 0.6843 - val_acc

In [12]:
inp_c = Input(shape=(SEQ_LEN, len(feature_cols)))
x_c = Conv1D(filters=64, kernel_size=3, padding='causal', activation='relu')(inp_c)
x_c = Conv1D(filters=64, kernel_size=3, padding='causal', activation='relu')(x_c)
x_c = LayerNormalization()(x_c)

attn_c = AttentionLayer()(x_c)
out_c = Dense(32, activation='relu')(attn_c)
out_c = Dropout(0.3)(out_c)
out_c = Dense(1, activation='sigmoid')(out_c)
model_cnn_attn = Model(inputs=inp_c, outputs=out_c)
model_cnn_attn.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])

callbacks3 = [
    ModelCheckpoint('cnn_attn_best.keras', save_best_only=True, monitor='val_accuracy'),
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=5)
]

In [13]:
history_cnn = model_cnn_attn.fit(
    X_train_s, y_train_s,
    validation_split=0.2, epochs=50, batch_size=16,
    callbacks=callbacks3, verbose=1
)

loss_ca, acc_ca = model_cnn_attn.evaluate(X_test_s, y_test_s)
print(f"CNN+Attn Test Acc: {acc_ca:.4f}")

Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.5830 - loss: 0.7305 - val_accuracy: 0.5000 - val_loss: 0.6951 - learning_rate: 0.0010
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4908 - loss: 0.7088 - val_accuracy: 0.5000 - val_loss: 0.6965 - learning_rate: 0.0010
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5002 - loss: 0.6998 - val_accuracy: 0.5000 - val_loss: 0.6933 - learning_rate: 0.0010
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4381 - loss: 0.7039 - val_accuracy: 0.5000 - val_loss: 0.6949 - learning_rate: 0.0010
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5232 - loss: 0.6907 - val_accuracy: 0.5000 - val_loss: 0.6945 - learning_rate: 0.0010
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5610 - loss: 0.6864 - val_accuracy: 0.5000 - val_loss: 0.6950 - learning_rate: 0.0010
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5690 - loss: 0.6859 - val_accur